# VectorStoreを利用したメモリ

`VectorStoreRetrieverMemory`は、メモリをVectorDBに格納し、呼び出されるたびに上位K個の最も「重要」なドキュメントを問い合わせます。

これは、他のほとんどのMemoryクラスと異なり、対話の順序を明示的に追跡しない点が異なります。

この場合 "docs " は以前の会話のスニペットです。これはAIが会話の中で以前に聞いた、関連のある情報の断片を参照するのに便利です。

In [1]:
from datetime import datetime
from langchain.embeddings.openai import OpenAIEmbeddings
from langchain.llms import OpenAI
from langchain.memory import VectorStoreRetrieverMemory
from langchain.chains import ConversationChain
from langchain.prompts import PromptTemplate

### VectorStoreを初期化する

選択したストアによって、このステップの内容は異なる場合があります。詳細については、関連するVectorStoreのドキュメントを参照してください。

In [2]:
import faiss

from langchain.docstore import InMemoryDocstore
from langchain.vectorstores import FAISS

# Dimensions of the OpenAIEmbeddings
# OpenAIEmbeddingsのディメンション
embedding_size = 1536
index = faiss.IndexFlatL2(embedding_size)
embedding_fn = OpenAIEmbeddings().embed_query
vectorstore = FAISS(embedding_fn, index, InMemoryDocstore({}), {})

### VectorStoreRetrieverMemory を作成する

メモリオブジェクトは、任意の VectorStoreRetriever からインスタンス化されます。

In [3]:
# 実際の使用では、`k`をより高い値に設定することになりますが、ここではk=1に設定して
# ベクトルルックアップが意味的に関連する情報を返すことを示します。
retriever = vectorstore.as_retriever(search_kwargs=dict(k=1))
memory = VectorStoreRetrieverMemory(retriever=retriever)

# メモリオブジェクトは、エージェントに追加すると、会話や使用したツールから適切な情報を保存することができます。
memory.save_context({"input": "好きな食べ物はピザです"}, {"output": "それはよかった"})
memory.save_context({"input": "好きなスポーツはサッカーです。"}, {"output": "..."})
memory.save_context({"input": "セルティックスはしない"}, {"output": "ok"}) #

In [4]:
# 最初に返された結果は、税務相談に関連する記憶であり、言語モデルは、
# 他のドキュメントよりも1099に意味的に関連していると判断しています（どちらも数字が含まれているにもかかわらず）。
print(memory.load_memory_variables({"prompt": "what sport should i watch?"})["history"])

input: 好きなスポーツはサッカーです。
output: ...


## チェーンでの使用
例を見ていきましょう。`verbose=True` を設定してプロンプトを表示できるようにします。

In [5]:
llm = OpenAI(temperature=0) # Can be any valid LLM
_DEFAULT_TEMPLATE = """The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know.

Relevant pieces of previous conversation:
{history}

(You do not need to use these pieces of information if not relevant)

Current conversation:
Human: {input}
AI:"""
PROMPT = PromptTemplate(
    input_variables=["history", "input"], template=_DEFAULT_TEMPLATE
)
conversation_with_summary = ConversationChain(
    llm=llm, 
    prompt=PROMPT,
    # テストのため、非常に低いmax_token_limitを設定しました。
    memory=memory,
    verbose=True
)

In [6]:
conversation_with_summary.predict(input="こんにちは、私の名前はペリーです、どうしましたか？")



> Entering new ConversationChain chain...
Prompt after formatting:
The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know.

Relevant pieces of previous conversation:
input: 好きな食べ物はピザです
output: それはよかった

(You do not need to use these pieces of information if not relevant)

Current conversation:
Human: こんにちは、私の名前はペリーです、どうしましたか？
AI:

> Finished chain.


' こんにちは、ペリーさん。私の名前はAIです。元気です。あなたは何をしていますか？'

In [8]:
# ここでは、バスケットボール関連のコンテンツを浮上させる
conversation_with_summary.predict(input="私の好きなスポーツは何ですか？")



> Entering new ConversationChain chain...
Prompt after formatting:
The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know.

Relevant pieces of previous conversation:
input: 好きなスポーツはサッカーです。
output: ...

(You do not need to use these pieces of information if not relevant)

Current conversation:
Human: 私の好きなスポーツは何ですか？
AI:

> Finished chain.


' 私にはわかりませんが、あなたが言ったように、あなたの好きなスポーツはサッカーですね。'

In [10]:
# 言語モデルがステートレス（状態を持たない）であっても、関連するメモリがフェッチされるため、時間について「推論」することができます。
# メモリやデータのタイムスタンプは、エージェントに時間的な関連性を判断させるために、一般的に有用である。
conversation_with_summary.predict(input="私の好きな食べ物は？")



> Entering new ConversationChain chain...
Prompt after formatting:
The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know.

Relevant pieces of previous conversation:
input: 私の好きな食べ物は？
response:  私にはそれを知る能力がありませんが、あなたの好きな食べ物は何ですか？

(You do not need to use these pieces of information if not relevant)

Current conversation:
Human: 私の好きな食べ物は？
AI:

> Finished chain.


' 私はあなたの好きな食べ物を知りませんが、あなたが好きな食べ物を教えてくれると嬉しいです！あなたの好きな食べ物は何ですか？'

In [11]:
# 会話からのメモリは自動的に保存されます。
# このクエリが上記のイントロダクションチャットと最もよく一致するため、
# エージェントはユーザーの名前を「覚えている」ことができます。
conversation_with_summary.predict(input="私の名前は何ですか？")



> Entering new ConversationChain chain...
Prompt after formatting:
The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know.

Relevant pieces of previous conversation:
input: こんにちは、私の名前はペリーです、どうしましたか？
response:  こんにちは、ペリーさん。私の名前はAIです。元気です。あなたは何をしていますか？

(You do not need to use these pieces of information if not relevant)

Current conversation:
Human: 私の名前は何ですか？
AI:

> Finished chain.


' あなたの名前はペリーさんですね。あなたは何をしていますか？'